## 千兆病理图像癌细胞转移检测
## Detecting Cancer Metastases on Gigapixel Pathology Images

In [1]:
ls

Detecting_Cancer_Metastases_on_Gigapixel_Pathology_Images.ipynb
Detecting_Cancer_metastases/
Mitosis_Detecting.ipynb
Mitosis_Detecting_v2.ipynb
camelyon/
data_preprocessing-latest.ipynb
data_preprocessing-nerd.ipynb
data_preprocessing.ipynb
data_preprocessing_of_camelyon/
data_processing.ipynb
download_Inception_V3.ipynb
retrain.ipynb


In [2]:
pwd

'/atlas/home/zwpeng/paper_rebuild'

In [3]:
import openslide
import numpy as np
from pylab import *

# 读取包含有肿瘤区域的大图（全切片病理图像）
origin_images_path = "/atlas/home/zwpeng/paper_rebuild/camelyon/train/tumor/origin_images/Tumor_005.tif"
origin_slide = openslide.open_slide(origin_images_path)

# 读取该肿瘤区域的标注图
annotation_images_path = "/atlas/home/zwpeng/paper_rebuild/camelyon/train/tumor/annotation_images/Tumor_005_Mask.tif"
mask_slide = openslide.open_slide(annotation_images_path)

In [4]:
# origin_slide.level_count,origin_slide.level_dimensions,origin_slide.level_downsamples     #查看病理图片的金字塔结构

In [5]:
# mask_slide.level_count,mask_slide.level_dimensions,mask_slide.level_downsamples     #查看标注图片的金字塔结构

### 首先找到感兴趣区域

In [6]:
level = 6
origin_size = origin_slide.dimensions
origin_widths = origin_size[0]
origin_heights = origin_size[1]

object_size = origin_slide.level_dimensions[level]
object_widths = int(object_size[0])
object_heights = object_size[1]

In [7]:
# 方法三：通过获取每一块区域的像素值R G B各自的平均数，然后相减，设置一个阈值，将噪点（墨迹）和有效区　分开

import time
from pylab import *
import numpy as np

start_time = time.clock()
level = 6

rgb_list_y = list()
rgb_list_x = list()
rgb_var_x = []
rgb_var_y = []
rgb_var_xi = []
rgb_var_yi = []

# 寻找有效区域的y值、高度
for k in range(100):
    slide = origin_slide.read_region((0, k*origin_heights//100), level, (object_widths, object_heights//50)) 
    slide_arr = array(slide.convert("RGB"))
    arrR = np.mean(slide_arr[:,:,:1])
    arrG = np.mean(slide_arr[:,:,1:2])
    arrB = np.mean(slide_arr[:,:,2:3])
    rgb_list_y.append((arrR,arrG,arrB))
for i,rgbVar in enumerate(rgb_list_y):
    rgb_var_y.append(np.var(rgbVar))
    if np.var(rgbVar)>=1:
        rgb_var_yi.append(i)

print(rgb_var_yi)
effective_y = min(rgb_var_yi)*origin_heights//100        #有效区域的左上顶点y坐标找到了
effective_heights = (max(rgb_var_yi)-min(rgb_var_yi))*origin_heights//100 + origin_heights//50  #有效区域的高度也出来了
print("有效区域的ｙ值是：%d" %effective_y, "有效区域的高度是：%d" %effective_heights)

# 寻找有效区域的x值、宽度
for j in range(100):
    slide = origin_slide.read_region((j*origin_widths//100, effective_y), level, 
                                      (object_widths//50, effective_heights//62))     # 循环顺序读取50宽的区域
#     slide = origin_slide.read_region((j*origin_widths//100, 0), level, 
#                                       (object_widths//50, object_heights))     # 循环顺序读取50宽的区域
    
    slide_arr = array(slide.convert("RGB"))
    arrR = np.mean(slide_arr[:,:,:1])
    arrG = np.mean(slide_arr[:,:,1:2])
    arrB = np.mean(slide_arr[:,:,2:3])
    rgb_list_x.append((arrR,arrG,arrB))
for i,rgbVar in enumerate(rgb_list_x):
    rgb_var_x.append(np.var(rgbVar))
    if np.var(rgbVar)>=2:
        rgb_var_xi.append(i)

print(rgb_var_xi)
effective_x = min(rgb_var_xi)*origin_widths//100        # 有效区域的左上顶点y坐标找到了
effective_widths = (max(rgb_var_xi) - min(rgb_var_xi))*origin_widths//100 + origin_widths//50  # 有效区域的宽度也出来了
print("有效区域的ｘ值是：%d" %effective_x, "有效区域的宽度是：%d" %effective_widths)
# plt.plot(range(100), rgb_var_y[:100], label='rgb_var_curve')
# plt.plot(range(100), rgb_var_x[:100], label='rgb_var_curve')
# plt.legend()
# plt.show()

end_time = time.clock()
print("本次程序运行时间是 %f 秒" %(end_time-start_time))

[57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78]
有效区域的ｙ值是：125199 有效区域的高度是：50518
[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
有效区域的ｘ值是：18580 有效区域的宽度是：41071
本次程序运行时间是 3.342803 秒


In [8]:
# 有效区域（感兴趣区域）
effective_area = (effective_x, effective_y)
effective_area_size = (effective_widths, effective_heights)
effective_area,effective_area_size

((18580, 125199), (41071, 50518))

这个有效区域，经过手动调试，起点坐标：(17600,124700) ，区域长宽：(44800,57600)



### mask 的有效区域，定位比较容易

In [9]:
from pylab import *
import numpy as np
mask_level = 7

# level0　的尺寸
mask_size = mask_slide.dimensions
mask_widths = mask_size[0]
mask_heights = mask_size[1]
# level7 的尺寸
mask_level_size = mask_slide.level_dimensions[mask_level]
mask_level_widths = mask_level_size[0]
mask_level_heights = mask_level_size[1]

mask_level_slide = mask_slide.read_region((0, 0), mask_level, (mask_level_widths, mask_level_heights))
mask_level_slide_gray = mask_level_slide.convert("L")
mask_level_slide_arr = array(mask_level_slide_gray)

mask_y, mask_x = nonzero(mask_level_slide_arr)  # 因为mask是黑白图，只需直接获得非零像素的坐标
# mask_x, mask_y

In [10]:
# 有效区域（感兴趣区域）level7上的
mask_effective_leftup_point = (min(mask_x), min(mask_y))
mask_effective_rightdown_point = (max(mask_x), max(mask_y))

mask_effective_leftup_point,mask_effective_rightdown_point

((451, 1066), (465, 1093))

In [11]:
mask_tumor_start = ((min(mask_x)-1) * int(mask_slide.level_downsamples[7]), 
                    (min(mask_y)-1) * int(mask_slide.level_downsamples[7]))
mask_tumor_end = ((max(mask_x)+1) * int(mask_slide.level_downsamples[7]), 
                  (max(mask_y)+1) * int(mask_slide.level_downsamples[7]))

mask_tumor_start, mask_tumor_end    # mask level0 图中tumor所在区域的起点和终点

((57600, 136320), (59648, 140032))

In [12]:
tumor_leftup_x = (min(mask_x)-1) * int(mask_slide.level_downsamples[7])
tumor_leftup_y = (min(mask_y)-1) * int(mask_slide.level_downsamples[7])
tumor_rightdown_x = (max(mask_x)+1) * int(mask_slide.level_downsamples[7])
tumor_rightdown_y = (max(mask_y)+1) * int(mask_slide.level_downsamples[7])
print(tumor_leftup_x,tumor_leftup_y,tumor_rightdown_x,tumor_rightdown_y)

57600 136320 59648 140032


In [13]:
mask_tumor_area = ((max(mask_x)-min(mask_x)+2)*int(mask_slide.level_downsamples[7]), 
                   (max(mask_y)-min(mask_y)+2)*int(mask_slide.level_downsamples[7]))
mask_tumor_area        # mask区域的长宽

(2048, 3712)

In [14]:
# mask_slide.read_region(mask_tumor_start, 0, mask_tumor_area) 
# mask_tumor_start 和　mask_tumor_area 这两个参数后面要用到

In [15]:
import time
import numpy as np
from PIL.Image import Image
from pylab import *
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

start_time = time.clock()

widths = 299
heights = 299

# 有效区域（感兴趣区域），所有细胞聚集区
# effective_area = (effective_x, effective_y)
# effective_area_size = (effective_widths, effective_heights)
# 有效区域，所有标注的细胞聚集区
# mask_slide.read_region(mask_tumor_start, 0, mask_tumor_area) 
# random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))  # 这一区域就是随机读到的区域，
                                                            #接下来就要判断这个区域在哪里，应该给他贴上什么标签

# 随机生成一个０到１之间的数，判断是否大于0.5,如果大于0.5,就从tumor区（mask）获取随机点（产生随机图片）；
# 如果小于0.5,就从normal区获取随机点，产生的随机图片需要判断是否取到了tumor
# def train_generator(widths=299, heights=299):
# for i in range(10):
def data_generator(widths=299,heights=299):
    while True:
        random_num = np.random.random(1)
        print(random_num)

        if random_num > 0.5:
            # 定义随机坐标,一定要取到一张含有tumor的图片
            random_x = np.random.randint(tumor_leftup_x, tumor_rightdown_x - widths)    # mask上的tumor有效区的起点和终点
            random_y = np.random.randint(tumor_leftup_y, tumor_rightdown_y - heights)
            print("取tumor随机点坐标是：%d,%d"%(random_x,random_y))
            random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
            random_img_mask_arr = array(random_img_mask.convert("L"))
            random__img_y, random_img_x = nonzero(random_img_mask_arr)
            while len(random_img_x)==0:
                random_x = np.random.randint(tumor_leftup_x, tumor_rightdown_x - widths)
                random_y = np.random.randint(tumor_leftup_y, tumor_rightdown_y - heights)
                print("取tumor随机点坐标是：%d,%d"%(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)

            #*********************上面这个 while 循环结束后，就产生了一个合格的坐标点*********************#
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))
    #         plt.imshow(random_img)
    #         plt.show()
            #***接下来就给他贴标签，并处理成训练所需的数据结构***#
            random_img_arr = array(random_img.convert("RGB"))
            x = np.expand_dims(random_img_arr, axis=0)/255.
            y = to_categorical(0,2)    
        else:
            # 定义随机坐标，一定要取到一张不含有tumor的normal图片
            random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)   # 大图上,nomal有效区的起点和终点
            random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
            print("取normal随机点坐标是：%d,%d"%(random_x,random_y))
            random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
            random_img_mask_arr = array(random_img_mask.convert("L"))
            random__img_y, random_img_x = nonzero(random_img_mask_arr)
            while len(random_img_x) != 0:
                random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)
                random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
                print("取normal随机点坐标是：%d,%d" %(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)

            #*********************上面这个 while 循环结束后，就产生了一个合格的坐标点*********************#
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))
    #         plt.imshow(random_img)
    #         plt.show()
            #***接下来就给他贴标签，并处理成训练所需的数据结构***#
            random_img_arr = array(random_img.convert("RGB"))
            x = np.expand_dims(random_img_arr, axis=0)/255.
            y = to_categorical(1,2) 
        yield (x,y)

Using TensorFlow backend.



接下来可以开始训练了。

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD


def Creat_InvepV3(num_lay_out=64,classes=2,dropout=0.5):
    img_width, img_height = 299, 299
    base_model = InceptionV3(weights="imagenet", include_top=False,
                             input_shape=(img_width, img_height, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(num_lay_out)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(classes)(x)
    x = BatchNormalization()(x)
    x = Activation('softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=SGD(lr=1e-4, momentum=0.4, decay=1e-6),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import SGD


def Creat_InvepV3(num_lay_out=1024,classes=2,dropout=0.5):
    img_width, img_height = 299, 299
    base_model = InceptionV3(weights="imagenet", include_top=False,
                             input_shape=(img_width, img_height, 3))
    x = base_model.output
    # x = Flatten()(x)
    # x = Dense(num_lay_out)(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Dropout(dropout)(x)
    # x = Dense(classes)(x)
    # x = BatchNormalization()(x)
    # x = Activation('softmax')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_lay_out, activation='relu')(x)
    predictions = Dense(classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=SGD(lr=1e-4, momentum=0.9),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
from __future__ import division
import keras
from keras.callbacks import CSVLogger
import gc

def train_model():
    keras.backend.tensorflow_backend.clear_session()
    epochs = 1
    model = Creat_InvepV3()
    csvlogger = CSVLogger('training.log', append=True)
    model.fit_generator(data_generator(),
                        steps_per_epoch=100,
                        epochs=epochs,
                        validation_data=data_generator(),
                        validation_steps=20,
                        verbose=0,
                        callbacks=[csvlogger])
    model.save('model.h5')
    gc.collect()
train_model()

[ 0.53591855]
取tumor随机点坐标是：57877,136942
取tumor随机点坐标是：59159,137311
取tumor随机点坐标是：59273,138226
取tumor随机点坐标是：58767,138587
[ 0.48391793]
取normal随机点坐标是：47285,128119
[ 0.52511982]
取tumor随机点坐标是：58667,138465
[ 0.50758334]
取tumor随机点坐标是：57915,139284
取tumor随机点坐标是：58030,138219
[ 0.25020268]
取normal随机点坐标是：25985,140715
[ 0.02454581]
取normal随机点坐标是：53522,154215
[ 0.51841799]
取tumor随机点坐标是：57754,136622
取tumor随机点坐标是：59169,139621
[ 0.85385961]
取tumor随机点坐标是：57944,137320
[ 0.03623745]
取normal随机点坐标是：59111,136020
[ 0.37847014]
取normal随机点坐标是：18922,160071
[ 0.2748491]
取normal随机点坐标是：34563,137889
[ 0.69319939]
取tumor随机点坐标是：58212,139142
[ 0.16805972]
取normal随机点坐标是：54370,161862
[ 0.02155465]
取normal随机点坐标是：43698,127639
[ 0.74484917]
取tumor随机点坐标是：59266,138904
[ 0.06389416]
取normal随机点坐标是：40127,156910
[ 0.71434665]
取tumor随机点坐标是：57675,136748
取tumor随机点坐标是：57946,136508
取tumor随机点坐标是：58391,137953
[ 0.36583053]
取normal随机点坐标是：40907,168710
[ 0.6819273]
取tumor随机点坐标是：58453,136338
[ 0.54186787]
取tumor随机点坐标是：58232,136650
[ 0.538363

In [ ]:
# samples = 16
# batch_size = 16
# nb_batch = samples//batch_size

    model.fit_generator(generator5.train_gen(path),
                        steps_per_epoch=1,
                        max_q_size=1,
                        epochs=epochs,
                        validation_data=None,
                        validation_steps=None,
                        verbose=1,
                        callbacks=[csvlogger])
    model.save('model.h5')
    gc.collect()
train_model('camelyon/')